<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/gridVeg_plant_functional_groups_WRANGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Security

* The user must load a `json` file containing the BigQuery API key into the local directory `/content/...`
* The user must have a Google Maps API key to enable mapping. 
   * CAUTION make sure the key is deleted from the current instance of the notebook before sharing

# Tools

In [2]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.5     ✔ dplyr   1.0.3
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



* Remember that the file containing authorization keys for Big Query must be loaded into the virutual envrionment manually.

In [1]:
install.packages("bigrquery")
library(bigrquery)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘gargle’, ‘rapidjsonr’




# Source

## Database Connection

In [3]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [4]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [5]:
billing <- bq_test_project()

## Database Query

### gridVeg_plant_intercepts

In [6]:
sql_plant_intercepts <- 
"
  SELECT 
    survey_ID,
    grid_point,
    key_plant_code,
    plant_native_status,
    plant_life_cycle,
    plant_life_form,
    intercepts_pct
  FROM 
    `mpg-data-warehouse.vegetation_gridVeg_summaries.gridVeg_foliar_cover_all`
"
bq_plant_intercepts <- bq_project_query(billing, sql_plant_intercepts)
tb_plant_intercepts <- bq_table_download(bq_plant_intercepts)
df_plant_intercepts <- as.data.frame(tb_plant_intercepts) %>% glimpse()

Rows: 23,857
Columns: 7
$ survey_ID           <chr> "436", "436", "436", "436", "436", "436", "436", …
$ grid_point          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ key_plant_code      <chr> "ELYTRA", "POAPAL", "FESCAM", "ARESER", "HEUCYL",…
$ plant_native_status <chr> "native", "native", "native", "nonnative", "nativ…
$ plant_life_cycle    <chr> "perennial", "perennial", "perennial", "annual", …
$ plant_life_form     <chr> "graminoid", "graminoid", "graminoid", "forb", "f…
$ intercepts_pct      <dbl> 2.5, 1.5, 14.5, 1.0, 2.5, 1.0, 1.0, 0.5, 0.5, 0.5…


### gridVeg_survey_metadata

In [7]:
sql_survey_metadata <- "
  SELECT 
    survey_ID,
    year,
    survey_sequence,
    grid_point
  FROM
    mpg-data-warehouse.vegetation_point_intercept_gridVeg.gridVeg_survey_metadata
"
bq_survey_metadata <- bq_project_query(billing, sql_survey_metadata)
tb_survey_metadata <- bq_table_download(bq_survey_metadata)
df_survey_metadata <- as.data.frame(tb_survey_metadata) %>% glimpse()

Rows: 1,472
Columns: 4
$ survey_ID       <chr> "F31C56A8-912D-410C-A17D-4C2DD75F71A4", "A19E87E6-A89…
$ year            <int> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016,…
$ survey_sequence <chr> "2016", "2016", "2016", "2016", "2016", "2016", "2016…
$ grid_point      <int> 30, 31, 32, 94, 153, 154, 155, 159, 160, 207, 215, 21…


# Wrangle

Start with the view vegetation_gridVeg_summaries:gridVeg_plant_intercepts. Remove records where key_plant_code = “NV” (corresponds with key_plant_species = 360).

## Explore key_plant_code "NV"

In [8]:
df_plant_intercepts %>%
  filter(key_plant_code == "NV") %>% 
  glimpse() 

Rows: 0
Columns: 7
$ survey_ID           <chr> 
$ grid_point          <int> 
$ key_plant_code      <chr> 
$ plant_native_status <chr> 
$ plant_life_cycle    <chr> 
$ plant_life_form     <chr> 
$ intercepts_pct      <dbl> 


## Remove key_plant_code "NV"

In [9]:
df_plant_functional_groups <- df_plant_intercepts %>%
  filter(key_plant_code != "NV")

## Calculate Detection Rate

Group the data on {survey_ID, plant_native_status, plant_life_cycle, plant_life_form}. Sum the intercepts_pct within each group, and divide the sums by 4 to obtain the detection_rate per 100 intercepts possible. 

In [11]:
df_plant_functional_groups %>%
  group_by(survey_ID, plant_native_status, plant_life_cycle, plant_life_form) %>%
  summarise(detection_rate = sum(intercepts_pct)) %>% ungroup() %>% 
  filter(survey_ID == "436")

`summarise()` has grouped output by 'survey_ID', 'plant_native_status', 'plant_life_cycle'. You can override using the `.groups` argument.



survey_ID,plant_native_status,plant_life_cycle,plant_life_form,detection_rate
<chr>,<chr>,<chr>,<chr>,<dbl>
436,native,annual,forb,3.5
436,native,perennial,forb,8.5
436,native,perennial,graminoid,44.5
436,native,perennial,shrub,10.5
436,native,perennial,tree,2.0
436,native,unknown,forb,1.5
436,nonnative,annual,forb,1.5
436,nonnative,annual,graminoid,5.5
436,nonnative,perennial,forb,0.5


In [12]:
df_detection_rate <- df_plant_functional_groups %>%
  group_by(survey_ID, plant_native_status, plant_life_cycle, plant_life_form) %>%
  summarise(detection_rate = sum(intercepts_pct)) %>% 
  ungroup()

`summarise()` has grouped output by 'survey_ID', 'plant_native_status', 'plant_life_cycle'. You can override using the `.groups` argument.



In [13]:
df_detection_rate %>% glimpse()

Rows: 8,998
Columns: 5
$ survey_ID           <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5FAD…
$ plant_native_status <chr> "native", "native", "native", "native", "native",…
$ plant_life_cycle    <chr> "annual", "perennial", "perennial", "perennial", …
$ plant_life_form     <chr> "forb", "forb", "graminoid", "shrub", "forb", "fo…
$ detection_rate      <dbl> 6.5, 14.5, 5.0, 8.5, 1.0, 5.0, 46.0, 25.0, 15.0, …


Then, make sure all combinations of functional groups that are found in the data are represented in each survey_ID. For those groups which were not detected at a survey_ID, fill the detection_rate with 0. This will complete the data set and make averages and other statistical comparisons more meaningful.

The number of records produced in the final dataset should be predictable from the number of surveys and the possible combinations of plant functional groups (pfg). With 1242 surveys and 25 pfg, we should end up with 31,050 records in the final data set.

In [14]:
# Number of surveys
df_detection_rate$survey_ID %>% 
  unique() %>% 
  length()

[1] 1242

In [15]:
# Possible combinations of pfg
df_detection_rate %>% 
  select(plant_native_status, plant_life_cycle, plant_life_form) %>% 
  distinct() %>% 
  arrange(plant_native_status, plant_life_cycle)

plant_native_status,plant_life_cycle,plant_life_form
<chr>,<chr>,<chr>
native,annual,forb
native,annual,graminoid
native,biennial,forb
native,perennial,forb
native,perennial,graminoid
native,perennial,shrub
native,perennial,tree
native,perennial,vine
native,unknown,forb


## Group Fill
Indeed, 31,050 records are produced. 

In [16]:
df_plant_functional_groups_comp <- 
  df_detection_rate %>%
    complete(survey_ID, 
            nesting(plant_native_status, plant_life_cycle, plant_life_form), 
            fill = list(detection_rate = 0)) %>% 
    glimpse()

Rows: 28,566
Columns: 5
$ survey_ID           <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "012C5FAD…
$ plant_native_status <chr> "native", "native", "native", "native", "native",…
$ plant_life_cycle    <chr> "annual", "annual", "biennial", "perennial", "per…
$ plant_life_form     <chr> "forb", "graminoid", "forb", "forb", "graminoid",…
$ detection_rate      <dbl> 6.5, 0.0, 0.0, 14.5, 5.0, 8.5, 0.0, 0.0, 1.0, 0.0…


## Join Metadata

Finally, join the metadata from requested tables or views and complete the schema below. 

In [17]:
# bring gridpoints back in
grid_point_ref <- df_plant_intercepts %>%
  select(survey_ID, grid_point) %>%
  distinct(survey_ID, grid_point)

In [18]:
df_plant_functional_groups_join <- 
  df_plant_functional_groups_comp %>%
  left_join(df_survey_metadata, by = "survey_ID") %>% 
  select(survey_ID, year, survey_sequence, grid_point, everything()) %>% 
  arrange(year, grid_point) %>% 
  glimpse()

Rows: 28,566
Columns: 8
$ survey_ID           <chr> "436", "436", "436", "436", "436", "436", "436", …
$ year                <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2…
$ survey_sequence     <chr> "2011-12", "2011-12", "2011-12", "2011-12", "2011…
$ grid_point          <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ plant_native_status <chr> "native", "native", "native", "native", "native",…
$ plant_life_cycle    <chr> "annual", "annual", "biennial", "perennial", "per…
$ plant_life_form     <chr> "forb", "graminoid", "forb", "forb", "graminoid",…
$ detection_rate      <dbl> 3.5, 0.0, 0.0, 8.5, 44.5, 10.5, 2.0, 0.0, 1.5, 0.…


# Output

In [19]:
# updated output 2021-01-26
write_csv(df_plant_functional_groups_join, file = "gridVeg_plant_functional_groups-WRANGLE.csv")

Warning message:
“The `path` argument of `write_csv()` is deprecated as of readr 1.4.0.
Please use the `file` argument instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”
